In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import leaguepedia_parser as lolp
import dataclasses

pd.set_option('display.min_rows', None)
plt.style.use("dark_background")

In [2]:
# Learning the methods for this API
dir(lolp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'get_all_team_assets',
 'get_game_details',
 'get_games',
 'get_long_team_name_from_trigram',
 'get_regions',
 'get_team_logo',
 'get_team_thumbnail',
 'get_tournaments',
 'parsers',
 'site',
 'transmuters']

In [3]:
lolp.get_tournaments()

[LeaguepediaTournament(name='LCK 2024 Season Opening', start='2024-01-09', end='2024-01-09', region='Korea', league=None, leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='LCK 2024 Season Opening'),
 LeaguepediaTournament(name='IWCT 2013', start='2013-08-21', end='2013-08-23', region='Wildcard', league='2014 International Wildcard Tournament', leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='Gamescom 2013/International Wildcard Tournament'),
 LeaguepediaTournament(name='IWCT 2014', start='2014-08-13', end='2014-09-01', region='Wildcard', league='2014 International Wildcard Tournament', leagueShort=None, rulebook=None, tournamentLevel='Primary', isQualifier=True, isPlayoffs=True, isOfficial=True, overviewPage='2014 Season International Wild Card Tournament'),
 LeaguepediaTournament(name='Brazil Regional Finals 2014', start='2014-07-18',

limit_output extension: Maximum message size of 1000 exceeded with 318603 characters

In [4]:
for a in lolp.get_tournaments():
    print(a.name)
    print(a.overviewPage)

LCK 2024 Season Opening
LCK 2024 Season Opening
IWCT 2013
Gamescom 2013/International Wildcard Tournament
IWCT 2014
2014 Season International Wild Card Tournament
Brazil Regional Finals 2014
CBLOL/2014 Season/Regional Finals
LAN 2015 Opening Cup
Latin America Cup 2015/LAN/Opening Cup/Regular Season
LAN 2016 Closing Cup Playoffs
Latin America Cup/LAN/2016 Season/Closing Cup/Playoffs
LAN 2016 Closing Cup
Latin America Cup/LAN/2016 Season/Closing Cup/Regular Season
LAN 2016 Opening Cup Playoffs
Latin America Cup/LAN/2016 Season/Opening Cup/Playoffs
LAN 2016 Opening Cup
Latin America Cup/LAN/2016 Season/Opening Cup/Regular Season
LAN 2015 Closing Cup Playoffs
Latin America Cup 2015/LAN/Closing Cup/Playoffs
LAN 2015 Opening Cup Playoffs
Latin America Cup 2015/LAN/Opening Cup/Playoffs
LAN 2015 Closing Cup
Latin America Cup 2015/LAN/Closing Cup/Regular Season
LAN 2016 Closing Cup Promotion
Latin America Cup/LAN/2016 Season/Closing Cup/Promotion
LAN 2016 Opening Cup Promotion
Latin America Cup

limit_output extension: Maximum message size of 1000 exceeded with 55474 characters

In [5]:
lolp.get_games("Intel Arabian Cup 2020/United Arab Emirates/Split 1")

[]

In [7]:
tournaments = lolp.get_tournaments()
for tournament in tournaments:
    if lolp.get_games(tournament.name) != []:
        print(tournament.name)
        print(lolp.get_games(tournament.name))
        print("------------------------------------------------------")

Intel Arabian Cup 2020
[LolGame(sources=EmptyDataclass(), duration=1524, start='2020-10-01T15:29:00+00:00', creation=None, type=None, queue_id=None, patch='10.20', gameVersion=None, winner='RED', teams=LolGameTeams(BLUE=LolGameTeam(bans=[223, 777, 84, 201, 235], players=[LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=54, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(primaryRuneTreeId=None, secondaryRuneTreeId=None, championId=120, id=None, inGameName=None, profileIconId=None, role=None, sources=EmptyDataclass(), runes=[], summonerSpells=[], endOfGameStats=None, snapshots=[], itemsEvents=[], wardsEvents=[], skillsLevelUpEvents=[], largeMonstersKills=[], levelUpEvents=[], spellsUses=[], specialKills=[]), LolGamePlayer(p

limit_output extension: Maximum message size of 1000 exceeded with 408837 characters

KeyboardInterrupt: 

In [ ]:
details = lolp.get_games("Intel Arabian Cup 2020")
test_dict = dataclasses.asdict(details[0])
test_dict

In [ ]:
print(test_dict["patch"])
print(test_dict["winner"])
print(test_dict["teams"]["BLUE"]["bans"])
for i in range(5):
    print(test_dict["teams"]["BLUE"]["players"][i]["championId"])
print(test_dict["teams"]["RED"]["bans"])
for i in range(5):
    print(test_dict["teams"]["RED"]["players"][i]["championId"])

In [ ]:
def list_banpicks(tournament):
    details = lolp.get_games(tournament)
    for i in range(len(details)):
        cur_dict = dataclasses.asdict(details[i])
        print(cur_dict["patch"])
        print(cur_dict["winner"])
        print(cur_dict["teams"]["BLUE"]["bans"])
        for i in range(5):
            print(cur_dict["teams"]["BLUE"]["players"][i]["championId"])
        print(cur_dict["teams"]["RED"]["bans"])
        for i in range(5):
            print(cur_dict["teams"]["RED"]["players"][i]["championId"])
        print("---------------------------------------------------")

In [ ]:
list_banpicks("2023 Season World Championship/Main Event")

In [ ]:
count=0
tournaments = lolp.get_tournaments()
for tournament in tournaments:
    if len(lolp.get_games(tournament.overviewPage)) != 0:
        count+=1

print(count)

In [ ]:
count=0
tournaments = lolp.get_tournaments()
for tournament in tournaments:
    if len(lolp.get_games(tournament.name)) != 0:
        print(tournament.name)

In [ ]:
t_with_data = []
tournaments = lolp.get_tournaments()
for tournament in tournaments:
    if len(lolp.get_games(tournament.overviewPage)) != 0:
        t_with_data.append(tournament.overviewPage)
print(t_with_data)

In [ ]:
list_banpicks('Season 3 World Championship')

In [ ]:
dataclasses.asdict(lolp.get_games('Season 3 World Championship')[0])["teams"]["RED"]["bans"]

In [ ]:
def store_banpicks(tournament):

    tournament_name = []
    date = []
    duration = []
    
    game_id = []
    champion_id = []
    patch = []
    side = []
    ban_pick = []
    win_lose = []
    details = lolp.get_games(tournament)
    for i in range(len(details)):
        cur_dict = dataclasses.asdict(details[i])
        
        # Blue side bans
        for j in range(len(cur_dict["teams"]["BLUE"]["bans"])):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["BLUE"]["bans"][j])
            side.append("BLUE")
            ban_pick.append("BAN")
            if cur_dict["winner"] == "BLUE":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
                
        # Red side bans
        for k in range(len(cur_dict["teams"]["RED"]["bans"])):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["RED"]["bans"][k])
            side.append("RED")
            ban_pick.append("BAN")
            if cur_dict["winner"] == "RED":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
                
        # Blue side picks
        for l in range(5):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["BLUE"]["players"][l]["championId"])
            side.append("BLUE")
            ban_pick.append("PICK")
            if cur_dict["winner"] == "BLUE":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")

        # Red side picks
        for m in range(5):
            game_id.append(i)
            tournament_name.append(tournament)
            date.append(cur_dict["start"][:10])
            duration.append(cur_dict["duration"])
            patch.append(cur_dict["patch"])
            
            champion_id.append(cur_dict["teams"]["RED"]["players"][m]["championId"])
            side.append("RED")
            ban_pick.append("PICK")
            if cur_dict["winner"] == "RED":
                win_lose.append("WIN")
            else:
                win_lose.append("LOSE")
        
    df = pd.DataFrame({"champion_id": champion_id, "patch": patch, "side": side, 
                        "ban_pick": ban_pick, "win_lose": win_lose, "tournament": tournament_name, 
                        "game_id": game_id, "date": date, "duration": duration})
        
    return df

In [ ]:
df_worlds_s3 = store_banpicks('Season 3 World Championship')
df_worlds_s3

In [ ]:
df = pd.DataFrame()
for t in t_with_data:
    cur_df = store_banpicks(t)
    df = df.append(cur_df)
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [ ]:
# can switch to show updated version
version = "13.11.1"
version_2 = "13.23.1"
champion_list = pd.read_json(f"https://ddragon.leagueoflegends.com/cdn/{version_2}/data/en_US/champion.json")

champion_list.drop(["type", "format", "version"], axis=1, inplace=True)
champion_list["id"] = ""

for champ in champion_list.index:
    champion_list["id"][champ] = int(champion_list.loc[champ, "data"]["key"])

champion_list.drop("data", axis=1, inplace=True)
champion_list = champion_list.reset_index().rename(columns={"index": "Champion"})
champion_list.to_csv("Champ_IDs.csv")

In [ ]:
df.patch.unique()

In [ ]:
df[df["patch"].isnull()==True].groupby(
    ["tournament"])["game_id"].count().reset_index()

In [ ]:
# I tried to find out what patch each tournament was played on for ones with missing patch info.
# Most were part of ongoing seasons that spanned multiple patches while the rest did not have available 
# info. For these cases, I have decided to automatically fill in patch info using the date of the game.

In [ ]:
patch_notes = pd.read_csv("lol_patch_notes.csv")
# for i in range(185):
#     patch_notes.Date[i] = patch_notes.Date[i][:-4]
# for i in range(185, 321):
#     patch_notes.Date[i] = patch_notes.Date[i][:-7]
# for i in range(321):
#     patch_notes.Date[i] = patch_notes.Date[i].replace('\n', ' ').strip()
# patch_notes.Date[257] = "June 17 2012"

# for converting str to datetime, pd.to_datetime converts to timestamp obj, which is faster 
# (and better than converting to dt.datetime obj)
# for i in range(321):
#     patch_notes.Date[i] = dt.strptime(patch_notes.Date[i], "%B %d %Y").date()
    
# patch_notes.Date = pd.to_datetime(patch_notes.Date, format="%B %d %Y")
patch_notes

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
# Info on the patch of tournaments that I found online.
update_patch = pd.DataFrame({"tournament": 
                              ['2012 MLG Pro Circuit/Fall/Championship',
                                '2014 Season Garena Regional Finals',
                                '2014 Season International Wild Card Tournament',
                                '2015 GPL Summer Playoffs',
                                'Battle of the Atlantic 2013',
                                'Gamescom 2013/International Wildcard Tournament',
                                'Latin America Cup 2015/LAN/Closing Cup/Promotion',
                                'Season 3 China Regional Finals',
                                'Season 3 Korea Regional Finals',
                                'Season 3 Latin America Regional Finals',
                                'Season 3 World Championship'], 
                              "patch": 
                              ['1.0.0.148',
                                '4.13',
                                '4.13',
                                '5.14',
                                '3.14',
                                '3.1',
                                '5.12',
                                '3.9',
                                '3.9',
                                '3.9',
                                '3.11']})
update_patch.to_csv("leaguepedia_some_missing_patch_info.csv")

In [ ]:
df3 = pd.DataFrame(df.groupby("patch")["game_id"].count()).reset_index()
for i in range(len(df3)):
    df3.loc[i, "patch"] = str(df3.loc[i, "patch"]) 

In [ ]:
pd.set_option('display.max_rows', None)
df.head(600)

In [ ]:
pd.DataFrame(df.groupby("patch")["game_id"].count()).reset_index()

In [ ]:
# update main df patch info with info found online.
df2 = df.copy()
for i in df2[df2["patch"].isnull()==True].index:
    for j in range(len(update_patch)):
        if df2.loc[i, "tournament"] == update_patch.loc[j, "tournament"]:
            df2.loc[i, "patch"] = update_patch.loc[j, "patch"]

In [ ]:
# update main df patch info by comparing game dates with patch dates (not necessarily accurate).

# took 34 minutes to execute
for i in df2[df2["patch"].isnull()==True].index:
    for j in range(len(patch_notes)):
        if df2.date[i] >= patch_notes.Date[j]:
            df2.loc[i, "patch"] = patch_notes.loc[j, "Patch"]
            break

In [ ]:
# drop rows with missing champion_id info (==0)
df2.drop(df2[df2["champion_id"]==0].index, axis=0, inplace=True)
df2.reset_index(inplace=True)
df2.drop(["index"], axis=1, inplace=True)

In [ ]:
# df2.to_csv("pick_bans_updated.csv")

In [ ]:
# Update the final pick_bans list
df_update = pd.DataFrame()
for t in t_with_data:
    if t not in df["tournament"].unique():
        cur_df = store_banpicks(t)
        df_update = pd.concat([df_update, cur_df])
df_update.reset_index(inplace=True)
df_update.drop("index", axis=1, inplace=True)

In [ ]:
# if champion_id == 0 for BAN, it means the team did not ban a champion for that slot.
df_update[df_update["champion_id"]==0]
df_update.drop(df_update[df_update["champion_id"]==0].index, axis=0, inplace=True)
df_update.reset_index(inplace=True)
df_update.drop(["index"], axis=1, inplace=True)
# df_update.to_csv("pick_bans_updated_2023nov26.csv")